In [ ]:
#Import librairies
import os
import jsonlines
from pymongo import MongoClient

folder_path = r"C:\Users\MSI\Desktop\P25\IF29\Projet_IF29_Tweets\.venv\data\tweet_worldcup_200twets"

# Connexion MongoDB
client = MongoClient("localhost", 27017)
db = client["IF29_twitter_db"]
collection = db["tweets_global"]

#  Importation des fichiers
file_list = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
total_files = len(file_list)

for i, filename in enumerate(file_list, start=1):
    file_path = os.path.join(folder_path, filename)
    try:
        with jsonlines.open(file_path) as reader:
            for line in reader:
                collection.insert_one(line)
    except Exception as e:
        print(f"Erreur lors de l import de {filename} : {e}")

client.close()

print(f"Importation des fichiers términés")

In [17]:

# Connexion à MongoDB
client = MongoClient("localhost", 27017)
db = client["IF29_twitter_db"]
tweets_collection = db["tweets_global"]

# Optionnel : Créer une nouvelle collection pour les utilisateurs
users_collection = db["user_db"]

# Parcourir tous les tweets
cursor = tweets_collection.find({}, {"user": 1})  # "user"

user_ids_seen = set()  # Pas les doublons

for doc in cursor:
    user = doc.get("user")
    if user and user["id_str"] not in user_ids_seen:
        user_ids_seen.add(user["id_str"])
        users_collection.insert_one(user)

print(f"{len(user_ids_seen)} profils utilisateurs uniques ont été extraits")
client.close()


KeyboardInterrupt: 

In [26]:
import pandas as pd
from pymongo import MongoClient
from datetime import datetime, timezone
import matplotlib.pyplot as plt
import seaborn as sns

# Connexion MongoDB
client = MongoClient("localhost", 27017)
db = client["IF29_twitter_db"]
user_col = db["twitter_users"]

# Chargement des utilisateurs
users = list(user_col.find())
df_users = pd.DataFrame(users)

# Conversion de la date de création des comptes
df_users["created_at"] = pd.to_datetime(
    df_users["created_at"], format="%a %b %d %H:%M:%S %z %Y", errors="coerce"
)

# Calcul de l age du compte
now_utc = datetime.utcnow().replace(tzinfo=timezone.utc)
df_users["account_age_years"] = (now_utc - df_users["created_at"]).dt.days / 365

# Calculs dérivés
df_users["followers_friends_ratio"] = df_users["followers_count"] / (df_users["friends_count"] + 1)
df_users["tweets_per_year"] = df_users["statuses_count"] / (df_users["account_age_years"] + 0.1)

# Nettoyage des valeurs 
df_users.replace([float('inf'), -float('inf')], pd.NA, inplace=True)
df_users.dropna(subset=["followers_friends_ratio", "account_age_years", "tweets_per_year"], inplace=True)

# Statistiques de base
display(df_users[[
    "followers_count", "friends_count", "statuses_count",
    "favourites_count", "followers_friends_ratio",
    "account_age_years", "tweets_per_year"
]].describe())

# Distribution du nombre de followers (log-log)
df_plot = df_users[df_users["followers_count"] > 0]

plt.figure(figsize=(10, 4))
sns.histplot(df_plot["followers_count"], bins=100, log_scale=(True, True), color="skyblue")
plt.title("Distribution du nombre de followers (log-log)")
plt.xlabel("Followers")
plt.ylabel("Nombre d'utilisateurs")
plt.show()

# Boxplot du ratio followers / friends
plt.figure(figsize=(10, 2))
sns.boxplot(x=df_users["followers_friends_ratio"], color="lightgreen")
plt.title("Boxplot du ratio followers / friends")
plt.show()


,followers_count,friends_count,statuses_count,favourites_count,followers_friends_ratio,account_age_years,tweets_per_year
count,2.674680e+05,267468.000000,2.674680e+05,2.674680e+05,2.674680e+05,267468.000000,267468.000000
mean,7.829827e+03,1101.990941,2.220438e+04,9.108702e+03,1.699535e+02,11.807005,1726.620049
std,1.817385e+05,5539.316636,5.474554e+04,2.527177e+04,1.217053e+04,2.853117,4050.877399
min,0.000000e+00,0.000000,1.000000e+00,0.000000e+00,0.000000e+00,6.989041,0.062175
25%,1.240000e+02,186.000000,1.329000e+03,3.800000e+02,3.745815e-01,9.268493,121.753072
50%,3.630000e+02,408.000000,6.275000e+03,1.946000e+03,8.455442e-01,12.180822,520.940929
75%,1.019000e+03,906.000000,2.175100e+04,7.547250e+03,1.555358e+00,14.115068,1722.075068
max,3.075537e+07,544830.000000,8.898917e+06,1.718515e+06,4.372115e+06,18.824658,490909.802010


C:\Users\MSI\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
